## AutoGen Tooling: Travel Tools Example

*[Coding along with the Udemy online course [Mastering AutoGen: Building Multi-Agent Systems](https://www.udemy.com/course/autogen-agent-systems/?couponCode=MTST7102224A2) by Paulo Dichone]*

AutoGen 2.0 for Python documentation @ https://microsoft.github.io/autogen/0.2/docs/Getting-Started

In [1]:
# https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/conversable_agent/
from autogen import ConversableAgent, UserProxyAgent
import pandas as pd

In [2]:
from typing import Annotated

Importing `autogen` returned the following error:<br/>
`flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.`

The temporary solution to solve this was to create a constraints.txt file with the following content:

`flaml==2.2.0`

Then running: `pip install -c constraints.txt autogen`

Following the suggestion at https://github.com/microsoft/autogen/issues/3548.

*FLAML @ https://microsoft.github.io/FLAML/ is "A Fast Library for Automated Machine Learning & Tuning."*

In [3]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [4]:
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0.0,
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [5]:
# emulate tools of a travel website
def get_flight_status(flight_number: Annotated[str, "Flight number"]) -> str:
    dummy_data = {"AA123": "On time", "DL456": "Delayed", "UA789": "Cancelled"}
    return f"The current status of flight {flight_number} id {dummy_data.get(flight_number)}."

def get_hotel_info(location: Annotated[str, "Location"]) -> str:
    dummy_data = {
        "New York": "Top hotel in New York: The Plaza - 5 Stars",
        "Los Angeles": "Top hotel in Los Angeles: Beverly Hills Hotel - 5 Stars",
        "Chicago": "Top hotel in Chicago: The Langham - 5 Stars"
    }
    return dummy_data.get(location, f"No hotels found in {location}.")


def get_travel_advice(location: Annotated[str, "Location"]) -> str:
    dummy_data = {
        "New York": "Travel advice for New York: Visit Central Park",
        "Los Angeles": "Travel advice for Los Angeles: Check out Hollywood Boulevard",
        "Chicago": "Travel advice for Chicago: Stroll the nature boardwalk around the stunning South Pond in Lincoln Park Zoo"
    }
    return dummy_data.get(location, f"No travel advice available for {location}.")

In [6]:
get_flight_status("AA123")

'The current status of flight AA123 id On time.'

In [7]:
get_hotel_info("New York")

'Top hotel in New York: The Plaza - 5 Stars'

In [8]:
get_travel_advice("Chicago")

'Travel advice for Chicago: Stroll the nature boardwalk around the stunning South Pond in Lincoln Park Zoo'

#### __Define the assistant agent that suggests tool calls__

In [9]:
assistant = ConversableAgent(
    name = "TravelAssistant",
    system_message = "You're a helpful AI travel assistant. Return 'TERMINATE' when the task is done.",
    llm_config = llm_config
)
assistant

#### __Define the user proxy agent__

In [10]:
user_proxy = ConversableAgent(
    name = "User",
    is_termination_msg = lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode = "NEVER"
)
user_proxy

#### __Register the tool signatures with the assistant agent__

In [11]:
assistant.register_for_llm(
    name="get_flight_status", 
    description="Get the current status of a flight based on the flight number."
)(get_flight_status)

assistant.register_for_llm(
    name="get_hotel_info", 
    description="Get information about hotels in a specific location."
)(get_hotel_info)

assistant.register_for_llm(
    name="get_travel_advice", 
    description="Get travel advice for a specific location."
)(get_travel_advice)

<function __main__.get_travel_advice(location: Annotated[str, 'Location']) -> str>

#### __Registering the tool functions with the user proxy agent__

In [12]:
user_proxy.register_for_execution(name="get_flight_status")(
    get_flight_status
)

user_proxy.register_for_execution(name="get_hotel_info")(
    get_hotel_info
)

user_proxy.register_for_execution(name="get_travel_advice")(
    get_travel_advice
)

<function __main__.get_travel_advice(location: Annotated[str, 'Location']) -> str>

#### __Initiate the chat__

In [13]:
user_proxy.initiate_chat(
    assistant,
    message = "What is the current status of flight AA123?"
)

User (to TravelAssistant):

What is the current status of flight AA123?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
TravelAssistant (to User):

***** Suggested tool call (call_XFTbFMZdHNy578BZ5ghwqemp): get_flight_status *****
Arguments: 
{"flight_number":"AA123"}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_flight_status...
User (to TravelAssistant):

User (to TravelAssistant):

***** Response from calling tool (call_XFTbFMZdHNy578BZ5ghwqemp) *****
The current status of flight AA123 id On time.
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
TravelAssistant (to User):

The current status of flight AA123 is "On time." 

TERMINATE

-----

ChatResult(chat_id=None, chat_history=[{'content': 'What is the current status of flight AA123?', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_XFTbFMZdHNy578BZ5ghwqemp', 'function': {'arguments': '{"flight_number":"AA123"}', 'name': 'get_flight_status'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'The current status of flight AA123 id On time.', 'tool_responses': [{'tool_call_id': 'call_XFTbFMZdHNy578BZ5ghwqemp', 'role': 'tool', 'content': 'The current status of flight AA123 id On time.'}], 'role': 'tool', 'name': 'User'}, {'content': 'The current status of flight AA123 is "On time." \n\nTERMINATE', 'role': 'user', 'name': 'TravelAssistant'}], summary='The current status of flight AA123 is "On time." \n\n', cost={'usage_including_cached_inference': {'total_cost': 7.005e-05, 'gpt-4o-mini-2024-07-18': {'cost': 7.005e-05, 'prompt_tokens': 327, 'completion_tokens': 35, 'total_tokens': 362}}, 'usage_excluding_cached_inference': {'total_cos

__A return message to this question without tooling looks like:__

User (to TravelAssistant):

What is the current status of flight AA123?

--------------------------------------------------------------------------------

> USING AUTO REPLY...
TravelAssistant (to User):

I'm sorry, but I don't have real-time flight status information. I recommend checking the airline's official website or a flight tracking service for the most up-to-date information on flight AA123. TERMINATE

--------------------------------------------------------------------------------
ChatResult(chat_id=None, chat_history=[{'content': 'What is the current status of flight AA123?', 'role': 'assistant', 'name': 'User'}, {'content': "I'm sorry, but I don't have real-time flight status information. I recommend checking the airline's official website or a flight tracking service for the most up-to-date information on flight AA123. TERMINATE", 'role': 'user', 'name': 'TravelAssistant'}], summary="I'm sorry, but I don't have real-time flight status information. I recommend checking the airline's official website or a flight tracking service for the most up-to-date information on flight AA123. ", cost={'usage_including_cached_inference': {'total_cost': 3.0299999999999998e-05, 'gpt-4o-mini-2024-07-18': {'cost': 3.0299999999999998e-05, 'prompt_tokens': 42, 'completion_tokens': 40, 'total_tokens': 82}}, 'usage_excluding_cached_inference': {'total_cost': 3.0299999999999998e-05, 'gpt-4o-mini-2024-07-18': {'cost': 3.0299999999999998e-05, 'prompt_tokens': 42, 'completion_tokens': 40, 'total_tokens': 82}}}, human_input=[])

In [16]:
user_proxy.initiate_chat(
    assistant,
    message = "I need help with my travel plans. Can you help me, I travel to Chicago? I need hotel information. And please check the status of my flight. The flight number is UA789"
)

User (to TravelAssistant):

I need help with my travel plans. Can you help me, I travel to Chicago? I need hotel information. And please check the status of my flight. The flight number is UA789

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
TravelAssistant (to User):

***** Suggested tool call (call_ERf8KI90s82q2sRXalIGi8SV): get_hotel_info *****
Arguments: 
{"location": "Chicago"}
*******************************************************************************
***** Suggested tool call (call_Jmzs84kAYTARbInaovFzKd9x): get_flight_status *****
Arguments: 
{"flight_number": "UA789"}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_hotel_info...

>>>>>>>> EXECUTING FUNCTION get_flight_status...
User (to TravelAssistant):

User (to TravelAssistant):

***** Response from calling t

ChatResult(chat_id=None, chat_history=[{'content': 'I need help with my travel plans. Can you help me, I travel to Chicago? I need hotel information. And please check the status of my flight. The flight number is UA789', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': 'call_ERf8KI90s82q2sRXalIGi8SV', 'function': {'arguments': '{"location": "Chicago"}', 'name': 'get_hotel_info'}, 'type': 'function'}, {'id': 'call_Jmzs84kAYTARbInaovFzKd9x', 'function': {'arguments': '{"flight_number": "UA789"}', 'name': 'get_flight_status'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'Top hotel in Chicago: The Langham - 5 Stars\n\nThe current status of flight UA789 id Cancelled.', 'tool_responses': [{'tool_call_id': 'call_ERf8KI90s82q2sRXalIGi8SV', 'role': 'tool', 'content': 'Top hotel in Chicago: The Langham - 5 Stars'}, {'tool_call_id': 'call_Jmzs84kAYTARbInaovFzKd9x', 'role': 'tool', 'content': 'The current status of flight UA789 id Cancelled.'}], 'role': 'to